In [1]:
import svetlanna as sv
import torch
from svetlanna.units import ureg
from svetlanna.visualization import show_specs, show_structure, show_stepwise_forward

# this helps to confirm system's compliance with physics
import warnings
warnings.filterwarnings('always')

Начнем с задания схемы с несколькими элементами:

In [ ]:
# параметры моделирования
Nx = 200
Ny = 200
sim_params = sv.SimulationParameters({
    'W': torch.linspace(-0.1 * ureg.cm, 0.1 * ureg.cm, Nx),
    'H': torch.linspace(-0.1 * ureg.cm, 0.1 * ureg.cm, Ny),
    'wavelength': 12 * ureg.um,
})

el1 = sv.elements.ThinLens(
    sim_params,
    focal_length=1 * ureg.cm,
    radius=2 * ureg.cm
)

el2 = sv.elements.FreeSpace(
    sim_params,
    distance=sv.Parameter(0.4*ureg.cm),
    method='AS'
)

el3 = sv.elements.ThinLens(
    sim_params,
    focal_length=sv.ConstrainedParameter(
        1 * ureg.cm,
        min_value=0.5 * ureg.mm,
        max_value=10 * ureg.cm
    ),
    radius=2 * ureg.cm
)

el4 = sv.elements.FreeSpace(
    sim_params,
    distance=sv.Parameter(0.2*ureg.cm),
    method='AS'
)

el5 = sv.elements.DiffractiveLayer(
    sim_params,
    mask=sv.ConstrainedParameter(
        torch.rand(Nx,Ny),
        min_value=0,
        max_value=2*torch.pi
    )
)

el6 = sv.elements.FreeSpace(
    sim_params,
    distance=sv.Parameter(0.4*ureg.cm),
    method='AS'
)

el7 = sv.elements.RoundAperture(
    sim_params,
    radius=0.03 * ureg.cm
)

el8 = sv.elements.FreeSpace(
    sim_params,
    distance=sv.Parameter(0.4*ureg.cm),
    method='AS'
)

el9 = sv.elements.Aperture(
    sim_params,
    mask=torch.rand(Nx,Ny)
)

In [3]:
setup = sv.LinearOpticalSetup(
    [el1, el2, el3, el4, el5, el6, el7, el8, el9]
)

В IPython вы можете проверить характеристики каждого элемента:

In [ ]:
el3  # Should show 'focal_length' and 'radius'

ThinLens(
  (focal_length_svtlnn_inner_parameter): InnerParameterStorageModule()
)

In [ ]:
el5  # должны отобразиться 'mask' и 'mask_norm'

DiffractiveLayer(
  (mask_svtlnn_inner_parameter): InnerParameterStorageModule()
)

Библиотека позволяет визуализировать всю настройку непосредственно в IPython:

In [10]:
show_structure(setup)

Вы также можете проверить характеристики каждого элемента, используя интерактивный виджет IPython:

In [11]:
show_specs(setup)

SpecsWidget(elements=[{'index': 0, 'name': 'LinearOpticalSetup', 'specs_html': '<div style="font-family:monosp…

Кроме того, можно визуализировать, как свет распространяется через каждый элемент в установке, используя другой интерактивный виджет IPython:

In [12]:
# wavefront to simulate
wavevefront = sv.Wavefront.gaussian_beam(sim_params, waist_radius=0.01 * ureg.cm)

show_stepwise_forward(setup, input=wavevefront, simulation_parameters=sim_params, types_to_plot=('A', 'Re', 'phase'))

StepwiseForwardWidget(elements=[{'index': 0, 'name': 'LinearOpticalSetup', 'output_image': 'iVBORw0KGgoAAAANSU…